In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,train_test_split
from sklearn.ensemble import RandomForestClassifier
# from sklearn.
import numpy as np

In [2]:
# Đọc tệp CSV vào DataFrame
df = pd.read_csv('mfcc_data_combined.csv')

# Trộn ngẫu nhiên các hàng của DataFrame
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

validation_data = df_shuffled.head(1000)
train_test_data = df_shuffled.drop(range(1000))

In [3]:
from collections import defaultdict

In [4]:
def find_model(data, models):  
    kf = KFold(n_splits=5, shuffle=False)
    y = data['label']
    X = data.drop(['label', 'file'], axis=1)

    # Khởi tạo một từ điển để lưu trữ tỉ lệ nhận dạng đúng cho từng nhãn
    label_accuracies = defaultdict(list)

    # Loop through models
    for model in models:
        
        acc_scores = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            # Khởi tạo model
            clf = model()

            # Train model
            clf.fit(X_train, y_train)

            # Dự đoán trên tập test
            y_pred = clf.predict(X_test)

            acc = accuracy_score(y_test, y_pred)
            acc_scores.append(acc)

            # Tính accuracy cho từng nhãn và lưu vào từ điển
            for label in y.unique():
                mask = y_test == label
                label_acc = accuracy_score(y_test[mask], y_pred[mask])
                label_accuracies[label].append(label_acc)

    # In ra tỉ lệ nhận dạng đúng cho từng nhãn
    for label, accs in label_accuracies.items():
        avg_acc = np.mean(accs)
        print(f"Label: {label}, Average Accuracy: {avg_acc}")
    
    avg_acc = np.mean(acc_scores)
    print(f"Model: {model.__name__}, Average Accuracy: {avg_acc}")

In [5]:
find_model(train_test_data, [RandomForestClassifier])

Label: Neutral, Average Accuracy: 0.5850736799765355
Label: Happy, Average Accuracy: 0.5548322058358257
Label: Fearful, Average Accuracy: 0.49151271811770336
Label: Disgusted, Average Accuracy: 0.5590659006524069
Label: Angry, Average Accuracy: 0.7867758566872528
Label: Suprised, Average Accuracy: 0.5989623721719903
Label: Sad, Average Accuracy: 0.804057396724905
Model: RandomForestClassifier, Average Accuracy: 0.6381221865763218
